    TO DO: HANDWERK ERUIT HALEN, NAMELIJK:
    - JAARLIJKS DE PROCESVARIABELE seizoen AANPASSEN
    - JAARLIJKS OPHALEN PROCESVARIABELE bestaande_gegevens aanpassen 

In [1]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import csv
from io import StringIO

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

#Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# proces variabelen, JAARLIJKS SEIZOEN AANPASSEN
# Bestaande gegevens ophalen (>2022, JAARLIJKS AANPASSEN) om nieuwe gegevens te kunnen bepalen
# kolomnamen van db ophalen, om ze waar nodig toe te voegen aan de csv bestanden met nieuwe gegevens
URL = 'https://www.football-data.co.uk/mmz4281'
seizoen = 2324
competities = ['E0','D1','I1','SP1','FR1','N1','B1','P1','T1','G1']
tel_aantal_nieuw = 0
aantal_al_aanwezig = 0
bestaande_gegevens = pd.read_sql('SELECT * FROM dsa.fd_stats WHERE cast(right(date,2) as int) >= extract(year from current_date) - 2001', sqal_connection)
kolomnamen_bestaand = bestaande_gegevens.columns.tolist()

# verwerken van nieuwe gegevens per competitie
for comp in competities:
        response = requests.get(f'{URL}/{seizoen}/{comp}.csv')
        csv_data = StringIO(response.text)
        nieuwe_gegevens = pd.read_csv(csv_data)
                
        # kolomnamen van de CSV bestanden naar lower case en HS en AS naar HSh en ASh, zodat het te matchen is met kolommen db
        nieuwe_gegevens.rename(columns={'HS':'hsh','AS':'ash'}, inplace=True)
        nieuwe_gegevens.columns = [naam.lower() for naam in nieuwe_gegevens.columns.tolist()]
        kolomnamen_nieuw = nieuwe_gegevens.columns
        
        # Als in db kolommen staan die ontbreken in CSV met nieuwe gegevens daar wordt deze kolom toegevoegd, zonder data
        # Dit is nodig om obv kolomnamen_bestaand de waarden op te halen in CSV met nieuwe gegevens, tbv INSERT
        for kolomnaam_bestaand in kolomnamen_bestaand: 
                if kolomnaam_bestaand not in kolomnamen_nieuw: nieuwe_gegevens[kolomnaam_bestaand] = None
        
        # loop door alle regels binnen een CSV bestand met nieuwe gegevens heen en tel het aantal nieuwe INSERTS
        for i, nieuw_gegeven in nieuwe_gegevens.iterrows():
                # maak voor iedere regel een lijst met alle waarden
                nieuw_gegeven_values = []
                for kolomnaam_bestaand in kolomnamen_bestaand:    
                        nieuw_gegeven_value = nieuw_gegeven[kolomnaam_bestaand]
                        nieuw_gegeven_values.append(nieuw_gegeven_value)
                
                # check of waarde al voorkomt in bestaande gegevens obv date en hometeam
                date = nieuw_gegeven['date']
                hometeam = nieuw_gegeven['hometeam']
                if not bestaande_gegevens[(bestaande_gegevens['date']==date) & (bestaande_gegevens['hometeam']==hometeam)].empty:
                        aantal_al_aanwezig += 1
                else:   
                        tel_aantal_nieuw += 1
                        psy_cursor.execute (
                                """
                                INSERT INTO DSA.fd_stats
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                                        %s
                                        );
                                DELETE from DSA.fd_stats WHERE date = 'NaN';
                                """
                        , nieuw_gegeven_values
                        )
                
print(f'aantal nieuw: {tel_aantal_nieuw}') 
print(f'aantal al aanwezig: {aantal_al_aanwezig}')         

# Commit de transactie en sluit de verbindingen
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close()

In [28]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import csv
from io import StringIO

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

#Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# proces variabelen, JAARLIJKS SEIZOEN AANPASSEN
# Bestaande gegevens ophalen (>2022, JAARLIJKS AANPASSEN) om nieuwe gegevens te kunnen bepalen
# kolomnamen van db ophalen, om ze waar nodig toe te voegen aan de csv bestanden met nieuwe gegevens
URL = 'https://www.football-data.co.uk/mmz4281'
seizoen = 2425
competities = ['E0']
tel_aantal_nieuw = 0
aantal_al_aanwezig = 0
bestaande_gegevens = pd.read_sql('SELECT * FROM dsa.fd_stats WHERE cast(right(date,2) as int) >= extract(year from current_date) - 2001', sqal_connection)
kolomnamen_bestaand = bestaande_gegevens.columns.tolist()

# verwerken van nieuwe gegevens per competitie
for comp in competities:
        response = requests.get(f'{URL}/{seizoen}/{comp}.csv')
        csv_data = StringIO(response.text)
        nieuwe_gegevens = pd.read_csv(csv_data)
                
        # kolomnamen van de CSV bestanden naar lower case en HS en AS naar HSh en ASh, zodat het te matchen is met kolommen db
        nieuwe_gegevens.rename(columns={'HS':'hsh','AS':'ash'}, inplace=True)
        nieuwe_gegevens.columns = [naam.lower() for naam in nieuwe_gegevens.columns.tolist()]
        kolomnamen_nieuw = nieuwe_gegevens.columns
        kolomnamen_nieuw.values[0] = 'div'
        print(kolomnamen_nieuw[0])


div


In [1]:
def connect_to_db():

    import sqlalchemy
    import sqlite3
    
    # Create SQL Alchemy database connection
    ##db_url = f"sqlite:///C:/Users/JanseDanny/OneDrive/Documenten Danny/Football Fortunes/database/DEV/footballfortunes.sqlite"
    db_url = f"sqlite:////app/data/database/DEV/footballfortunes.sqlite"
    engine = sqlalchemy.create_engine(db_url, echo=True)
    sqal_connection = engine.connect()

    # Create Psychop connection
    ##psy_connection = sqlite3.connect("C:/Users/JanseDanny/OneDrive/Documenten Danny/Football Fortunes/database/DEV/footballfortunes.sqlite")
    psy_connection = sqlite3.connect("/app/data/database/DEV/footballfortunes.sqlite")
    
    return psy_connection, sqal_connection

def close_connection_db(psy_connection, sqal_connection):
    
    # Commit de transactie en sluit de verbinding
    sqal_connection.close()
    psy_connection.close()  